## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 

Sarving and Death is taken care of elsewhere, so this notebook won't have that filter. 


In [1]:
import sys
import numpy as np
import pandas as pd

# mapping to the modules that make the app
sys.path.insert(0, "../..")



import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

#importing the libraries from the app
from app.connectors import cmdb_graph
from app.objects import time
from app.objects import population, species
from app.functions import language, maths

c = cmdb_graph.CosmosdbClient()


executing local windows deployment


Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [3]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2024-04-21T18:58:16.417572+00:00 UTU:13989 >

In [4]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 13989}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [5]:
params['pop_health_requirement'] = 0.4

In [6]:
healthy_pops_query = f"""
        g.V().has('label','pop')
            .has('health',gt({params.get("pop_health_requirement")})).as('pop')
            .values('objid','health','wealth')
        """

c.run_query(healthy_pops_query)
c.res

['4659716792872',
 0.7,
 0.651,
 '9423196100160',
 0.7,
 0.5328,
 '4753859535188',
 0.7,
 0.5143,
 '3660563754665',
 0.7,
 0.5828,
 '2909803569658',
 0.7,
 0.522,
 '8603380707717',
 0.7,
 0.5645,
 '0253192577182',
 0.7,
 0.6035,
 '6648922593970',
 0.7,
 0.3835,
 '1977798732964',
 0.7,
 0.5315,
 '8156212511459',
 0.7,
 0.4875,
 '0804013652458',
 0.7,
 0.4728,
 '5840971700979',
 0.7,
 0.5397,
 '3276597990265',
 0.7,
 0.5348,
 '2797495206916',
 0.7,
 0.5662]

chop those results into a dataframe.

In [7]:
a = np.array(c.res)
np.split(a,len(a)/3)
pops_df = pd.DataFrame(np.split(a,len(a)/3),columns=['objid','health','wealth'])
pops_df[['health','wealth']] = pops_df[['health','wealth']].astype(float)
pops_df

,objid,health,wealth
0,4659716792872,0.7,0.6510
1,9423196100160,0.7,0.5328
2,4753859535188,0.7,0.5143
3,3660563754665,0.7,0.5828
4,2909803569658,0.7,0.5220
5,8603380707717,0.7,0.5645
6,0253192577182,0.7,0.6035
7,6648922593970,0.7,0.3835
8,1977798732964,0.7,0.5315
9,8156212511459,0.7,0.4875


Add randomness

In [8]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df

,objid,health,wealth,roll
0,4659716792872,0.7,0.6510,0.739809
1,9423196100160,0.7,0.5328,0.053943
2,4753859535188,0.7,0.5143,0.788845
3,3660563754665,0.7,0.5828,0.391167
4,2909803569658,0.7,0.5220,0.771922
5,8603380707717,0.7,0.5645,0.203556
6,0253192577182,0.7,0.6035,0.137790
7,6648922593970,0.7,0.3835,0.717699
8,1977798732964,0.7,0.5315,0.342138
9,8156212511459,0.7,0.4875,0.867895


select the populations that will grow. 

In [9]:
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']
pops_df

,objid,health,wealth,roll,grow
0,4659716792872,0.7,0.6510,0.739809,False
1,9423196100160,0.7,0.5328,0.053943,True
2,4753859535188,0.7,0.5143,0.788845,False
3,3660563754665,0.7,0.5828,0.391167,True
4,2909803569658,0.7,0.5220,0.771922,False
5,8603380707717,0.7,0.5645,0.203556,True
6,0253192577182,0.7,0.6035,0.137790,True
7,6648922593970,0.7,0.3835,0.717699,False
8,1977798732964,0.7,0.5315,0.342138,True
9,8156212511459,0.7,0.4875,0.867895,False


In [10]:
reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1).reset_index(drop=True)
reproducing_pops

,objid,health,wealth
0,9423196100160,0.7,0.5328
1,3660563754665,0.7,0.5828
2,8603380707717,0.7,0.5645
3,0253192577182,0.7,0.6035
4,1977798732964,0.7,0.5315
5,0804013652458,0.7,0.4728
6,3276597990265,0.7,0.5348
7,2797495206916,0.7,0.5662


Now that we have the list of reproducing pops, we can generate new species from them. 


In [11]:
parent_pop = reproducing_pops.sample().to_dict(orient='records')[0]
parent_pop

{'objid': '3660563754665', 'health': 0.7, 'wealth': 0.5828}

Note, there are a specific list of edges that a child note must have. 

In [12]:
query_child_must_have = f"""
        g.V().has('objid','{parent_pop["objid"]}')
            .outE().values('label','inVLabel','outVLabel')
        """


c.run_query(query_child_must_have)
required_edges = c.res
required_edges

['isIn', 'isOf', 'inhabits']

* 'isOf' - The Species
* 'inhabits' - planet or vessal that it is in
* 'childOf' - It's parent pop
* 'isIn' - The faction that it is in. 

In [13]:
query_pop_species_faction = f"""
        g.V().has('objid','{parent_pop["objid"]}')
            .local(
                union(
                    out('inhabits').as('location'),
                    out('isOf').as('species'),
                    out('isIn').as('faction')
                    )
                    .fold()).as('pop','location','species','faction')
                .path()
                .by(unfold().valueMap().fold())
        """


c.run_query(query_pop_species_faction)
c.res

[{'labels': [[], ['pop', 'location', 'species', 'faction']],
  'objects': [[{'objid': ['3660563754665'],
     'name': ['Luling Pegashia'],
     'conformity': [0.491],
     'literacy': [0.604],
     'aggression': [0.625],
     'constitution': [0.498],
     'health': [0.7],
     'isIn': ['0697488714945'],
     'industry': [0.5615],
     'wealth': [0.5828],
     'factionLoyalty': [0.538],
     'isIdle': ['true'],
     'userguid': ['ac5b8081-7ef9-4bce-baac-6d0ea7e1782c'],
     'objtype': ['pop']}],
   [{'name': ['Sar'],
     'class': ['terrestrial'],
     'objid': ['6430884923575'],
     'radius': [0.508],
     'mass': [0.967],
     'orbitsDistance': [0.813],
     'orbitsId': ['6701681243146'],
     'orbitsName': ['Barmasgranmi'],
     'isSupportsLife': ['true'],
     'isPopulated': ['true'],
     'isHomeworld': ['true'],
     'userguid': ['ac5b8081-7ef9-4bce-baac-6d0ea7e1782c'],
     'objtype': ['planet']},
    {'name': ['Le'],
     'objid': ['4284334697626'],
     'consumes': ['organics'

In [14]:
pop_dict = c.clean_node(c.res[0]['objects'][0][0])
pop_dict

{'objid': '3660563754665',
 'name': 'Luling Pegashia',
 'conformity': 0.491,
 'literacy': 0.604,
 'aggression': 0.625,
 'constitution': 0.498,
 'health': 0.7,
 'isIn': '0697488714945',
 'industry': 0.5615,
 'wealth': 0.5828,
 'factionLoyalty': 0.538,
 'isIdle': 'true',
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'pop',
 'id': '3660563754665'}

In [15]:
loc_dict = c.clean_node(c.res[0]['objects'][1][0])
loc_dict

{'name': 'Sar',
 'class': 'terrestrial',
 'objid': '6430884923575',
 'radius': 0.508,
 'mass': 0.967,
 'orbitsDistance': 0.813,
 'orbitsId': '6701681243146',
 'orbitsName': 'Barmasgranmi',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'planet',
 'id': '6430884923575'}

In [16]:
sp_dict = c.clean_node(c.res[0]['objects'][1][1])
sp_dict

{'name': 'Le',
 'objid': '4284334697626',
 'consumes': 'organics',
 'effuses': 'organic waste,plastics',
 'conformity': '0.5',
 'aggression': '0.5',
 'literacy': '0.5',
 'constitution': '0.5',
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'species',
 'id': '4284334697626'}

In [17]:
fact_dict = c.clean_node(c.res[0]['objects'][1][2])
fact_dict

{'name': 'Luling',
 'objid': '0697488714945',
 'lat': -0.096,
 'long': 0.0,
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'faction',
 'id': '0697488714945'}

Note that the formatting (created by the structure of the query is)
```
[
    [pop],
    [species,faction]
]
```

Going to create species objects for the populations that will reproduce

In [18]:
sp = species.Species(sp_dict)
sp

<species: None; 4284334697626; Le>

Loading in the parent attributes as defaults

In [19]:
sp.config['defaults'] = pop_dict

In [20]:
sp.config

{'name': 'Le',
 'objid': '4284334697626',
 'consumes': 'organics',
 'effuses': 'organic waste,plastics',
 'conformity': '0.5',
 'aggression': '0.5',
 'literacy': '0.5',
 'constitution': '0.5',
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'species',
 'id': '4284334697626',
 'defaults': {'objid': '3660563754665',
  'name': 'Luling Pegashia',
  'conformity': 0.491,
  'literacy': 0.604,
  'aggression': 0.625,
  'constitution': 0.498,
  'health': 0.7,
  'isIn': '0697488714945',
  'industry': 0.5615,
  'wealth': 0.5828,
  'factionLoyalty': 0.538,
  'isIdle': 'true',
  'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
  'objtype': 'pop',
  'id': '3660563754665'}}

In [21]:
child = population.Pop(sp)
child.name = sp.config['defaults']['name']+language.make_word(1).lower()
child

<pop: pop; 3650739535522; Luling Pegashiaen>

Note that the child must have a userguid

In [22]:
child.get_data()

{'name': 'Luling Pegashiaen',
 'objid': '3650739535522',
 'label': 'pop',
 'conformity': 0.615,
 'literacy': 0.486,
 'aggression': 0.441,
 'constitution': 0.534,
 'health': 0.7,
 'isIn': None,
 'industry': 0.48750000000000004,
 'wealth': 0.48675,
 'factionLoyalty': 0.542,
 'isIdle': True}

Now that I have the child population, I can check that they have the needed edges. 

In [23]:
required_edges

['isIn', 'isOf', 'inhabits']

In [24]:
child.childOf

{'node1': '3650739535522', 'node2': '3660563754665', 'label': 'childOf'}

In [25]:
child.isOfSpecies

{'node1': '3650739535522', 'node2': '4284334697626', 'label': 'isOf'}

In [26]:
isIn_edge = child.get_isInFaction()
isIn_edge['node2'] = fact_dict['objid']
isIn_edge

{'node1': '3650739535522', 'node2': '0697488714945', 'label': 'isIn'}

In [27]:
inhabits_edge = {'node1': child.objid, 'node2': loc_dict['objid'], 'label': 'inhabits'}
inhabits_edge

{'node1': '3650739535522', 'node2': '6430884923575', 'label': 'inhabits'}

In [28]:
pd.DataFrame([child.childOf,
              child.isOfSpecies,
              isIn_edge,
              inhabits_edge])

,node1,node2,label
0,3650739535522,3660563754665,childOf
1,3650739535522,4284334697626,isOf
2,3650739535522,0697488714945,isIn
3,3650739535522,6430884923575,inhabits


## Building the uploaded data

In [29]:
child_data = child.get_data()
child_data['userguid'] = pop_dict['userguid'] 
child_data

{'name': 'Luling Pegashiaen',
 'objid': '3650739535522',
 'label': 'pop',
 'conformity': 0.615,
 'literacy': 0.486,
 'aggression': 0.441,
 'constitution': 0.534,
 'health': 0.7,
 'isIn': None,
 'industry': 0.48750000000000004,
 'wealth': 0.48675,
 'factionLoyalty': 0.542,
 'isIdle': True,
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c'}

In [30]:
def population_growth_event(t,parent,location,child):
    node = {
        'objid':maths.uuid(),
        'name':'population growth',
        'label':'event',
        'text': f"The population ({parent['name']}) inhabiting {location['name']} has grown to produce the population: {child.name}.",
        'visibleTo':parent['userguid'],
        'time':t.params['currentTime'],
        'userguid':parent['userguid'],
        'source':'notebook'
    }
    return node

event = population_growth_event(t,pop_dict,loc_dict,child)
event

{'objid': '7255912950476',
 'name': 'population growth',
 'label': 'event',
 'text': 'The population (Luling Pegashia) inhabiting Sar has grown to produce the population: Luling Pegashiaen.',
 'visibleTo': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'time': 13989,
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'source': 'notebook'}

In [31]:
event_edge = {'node1': child.objid, 'node2': event['objid'], 'label': 'caused'}

In [32]:
data = {"nodes":[child_data, event] ,"edges":[inhabits_edge,child.isOfSpecies,isIn_edge,event_edge] }

print(f"The final dataset is {len(data.get('nodes'))} nodes and {len(data.get('edges'))} edges")

The final dataset is 2 nodes and 4 edges


In [33]:
pd.DataFrame(data['edges'])

,node1,node2,label
0,3650739535522,6430884923575,inhabits
1,3650739535522,4284334697626,isOf
2,3650739535522,0697488714945,isIn
3,3650739535522,7255912950476,caused


In [34]:
pd.DataFrame(data['nodes']) 

,name,objid,label,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,isIdle,userguid,text,visibleTo,time,source
0,Luling Pegashiaen,3650739535522,pop,0.615,0.486,0.441,0.534,0.7,NaN,0.4875,0.48675,0.542,True,ac5b8081-7ef9-4bce-baac-6d0ea7e1782c,NaN,NaN,NaN,NaN
1,population growth,7255912950476,event,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ac5b8081-7ef9-4bce-baac-6d0ea7e1782c,The population (Luling Pegashia) inhabiting Sa...,ac5b8081-7ef9-4bce-baac-6d0ea7e1782c,13989.0,notebook


It takes a while to upload all of that data. 

In [35]:
# c.upload_data('notebook',data)